In [1]:
import pandas as pd
import numpy as np
import datetime
import os

os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q3/Planner_request/Bruce_Margret_20190815_Furniture_2019_Q1_Q2'

In [2]:
def recursive_file_gen(root_path):
    for root, dirs, files in os.walk(root_path):
        for file in files:
            yield os.path.join(root, file)

In [7]:
all_2019_daily_sales_files=list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
all_2019_daily_sales_files=[x for x in all_2019_daily_sales_files if "daily" in x.lower()]

week_end_dts_2019Q1=[str(datetime.date(2019,2,9)+datetime.timedelta(days=7*x)) for x in range(13)]
week_end_dts_2019Q2=[str(datetime.date(2019,2,9)+datetime.timedelta(days=13*7+7*x)) for x in range(13)]

In [22]:
list_2019Q1_files=[]
for week_end_dt in week_end_dts_2019Q1:
    list_2019Q1_files.extend([x for x in all_2019_daily_sales_files if week_end_dt in x])
list_2019Q1_files=list_2019Q1_files[1:]
list_2019Q1_files.extend(["/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20190209.txt"])



list_2019Q2_files=[]
for week_end_dt in week_end_dts_2019Q2:
    list_2019Q2_files.extend([x for x in all_2019_daily_sales_files if week_end_dt in x])

In [25]:
df_division_name=pd.read_table("/home/jian/BigLots/static_files/MediaStorm Data Extract - Division Names.txt",sep="|",dtype=str)
df_division_name



,division_id,division_desc
0,1,FOOD
1,2,CONSUMABLES
2,3,SEASONAL
3,4,SOFTHOME
4,5,HARDHOME
5,6,ELECNACCES
6,7,Division 7
7,8,DIVISION 8 (EMP DISCOUNT)
8,9,FURNITURE
9,10,DIVISION 10


In [31]:
df_class_division_mapping=pd.read_table("/home/jian/BigLots/static_files/ProductTaxonomy/MediaStormProductTaxonomy20190801-135940-477.txt",sep="|",dtype=str)
df_class_division_mapping=df_class_division_mapping[['division_id','class_code_id']].drop_duplicates()
df_class_division_mapping.shape

furniture_classcode_list=df_class_division_mapping[df_class_division_mapping['division_id']=="9"]['class_code_id'].tolist()

In [39]:
df_2019_furniture_Q1=pd.DataFrame()
for file in list_2019Q1_files:
    df=pd.read_table(file,sep="|",dtype=str,
                     usecols=['location_id','transaction_dt','customer_id_hashed','class_code_id','item_transaction_amt'])
    df_date_min=df['transaction_dt'].min()
    df_date_max=df['transaction_dt'].max()
    
    df=df[df['class_code_id'].isin(furniture_classcode_list)]
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df['rewards_level']=np.where(pd.isnull(df['customer_id_hashed']),"Non_Rewards","Rewards")
    
    df=df.groupby(["location_id",'rewards_level','class_code_id'])['item_transaction_amt'].sum().to_frame().reset_index()
    df['week_start_dt']=df_date_min
    df['week_start_dt']=df_date_max
    df_2019_furniture_Q1=df_2019_furniture_Q1.append(df)
    print(df_date_min,df_date_max,datetime.datetime.now())
    
df_2019_furniture_Q1['store_type']=np.where(df_2019_furniture_Q1['location_id']=="6990","Online","Instore")

2019-02-10 2019-02-16 2019-08-15 14:51:07.900012
2019-02-17 2019-02-23 2019-08-15 14:52:51.634600
2019-02-24 2019-03-02 2019-08-15 14:56:44.050658
2019-03-03 2019-03-09 2019-08-15 15:01:29.735097
2019-03-10 2019-03-16 2019-08-15 15:06:30.382819
2019-03-17 2019-03-23 2019-08-15 15:10:32.639386
2019-03-24 2019-03-30 2019-08-15 15:13:50.974995
2019-03-31 2019-04-06 2019-08-15 15:17:49.062433
2019-04-07 2019-04-13 2019-08-15 15:21:25.697553
2019-04-14 2019-04-20 2019-08-15 15:24:41.129315
2019-04-21 2019-04-27 2019-08-15 15:27:24.656239
2019-04-28 2019-05-04 2019-08-15 15:33:08.202608
2019-02-03 2019-02-09 2019-08-15 15:36:26.531583


In [40]:
df_2019_furniture_Q2=pd.DataFrame()
for file in list_2019Q2_files:
    df=pd.read_table(file,sep="|",dtype=str,
                     usecols=['location_id','transaction_dt','customer_id_hashed','class_code_id','item_transaction_amt'])
    df_date_min=df['transaction_dt'].min()
    df_date_max=df['transaction_dt'].max()
    
    df=df[df['class_code_id'].isin(furniture_classcode_list)]
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df['rewards_level']=np.where(pd.isnull(df['customer_id_hashed']),"Non_Rewards","Rewards")
    
    df=df.groupby(["location_id",'rewards_level','class_code_id'])['item_transaction_amt'].sum().to_frame().reset_index()
    df['week_start_dt']=df_date_min
    df['week_start_dt']=df_date_max
    df_2019_furniture_Q2=df_2019_furniture_Q2.append(df)
    print(df_date_min,df_date_max,datetime.datetime.now())
    
df_2019_furniture_Q2['store_type']=np.where(df_2019_furniture_Q2['location_id']=="6990","Online","Instore")

2019-05-05 2019-05-11 2019-08-15 15:40:38.842274
2019-05-12 2019-05-18 2019-08-15 15:43:23.075507
2019-05-19 2019-05-25 2019-08-15 15:47:42.509668
2019-05-26 2019-06-01 2019-08-15 15:52:26.736751
2019-06-02 2019-06-08 2019-08-15 15:56:14.369406
2019-06-09 2019-06-15 2019-08-15 15:59:50.699140
2019-06-16 2019-06-22 2019-08-15 16:02:07.156722
2019-06-23 2019-06-29 2019-08-15 16:05:06.550732
2019-06-30 2019-07-06 2019-08-15 16:07:57.859047
2019-07-07 2019-07-13 2019-08-15 16:11:23.399472
2019-07-14 2019-07-20 2019-08-15 16:12:38.801733
2019-07-21 2019-07-27 2019-08-15 16:13:26.991259
2019-07-28 2019-08-03 2019-08-15 16:14:03.048979


In [45]:
df_2019Q1_furniture=df_2019_furniture_Q1.groupby(["store_type","rewards_level",'class_code_id'])['item_transaction_amt'].sum().to_frame().reset_index()
df_2019Q2_furniture=df_2019_furniture_Q2.groupby(["store_type","rewards_level",'class_code_id'])['item_transaction_amt'].sum().to_frame().reset_index()

In [49]:
writer=pd.ExcelWriter("./BL_2019_Q1_Q2_furniture_sales_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_2019Q1_furniture.to_excel(writer,"2019_Q1_Furniture",index=False)
df_2019Q2_furniture.to_excel(writer,"2019_Q2_Furniture",index=False)
writer.save()

In [48]:
df_2019_furniture_Q1.to_csv("./df_2019_furniture_Q1.csv",index=False)
df_2019_furniture_Q2.to_csv("./df_2019_furniture_Q2.csv",index=False)